# Sources

sources:

    https://nanonets.com/blog/langchain/#module-i-model-io                               langchain blog
    https://python.langchain.com/docs/integrations/platforms/openai/    langchain  openai doc related fun of openai
    https://kleiber.me/blog/2023/05/14/tracking-inspecting-prompts-langchain-agents-weights-and-biases/      best blog for inspection
    https://nanonets.com/blog/langchain/#module-iii-agents
    https://www.pinecone.io/learn/series/langchain/langchain-tools/        custom tool creation
    https://medium.com/data-science-in-your-pocket/mathematics-using-llms-using-langchains-ca23bbd1a38b              llm math



# Imports

In [4]:
#pip install langchain-openai
#pip install langchain --upgrade
#pip install numexpr
#pip install langchainhub


from langchain.agents import initialize_agent
from langchain_openai import OpenAI
from langchain.agents import load_tools
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings

# set env variabel

In [5]:
import os
OPENAI_KEY = os.getenv("MY_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_KEY


In [272]:

template = """tell me the colore of {frute} in single word."""

prompt = PromptTemplate.from_template(template)

In [273]:
prompt

PromptTemplate(input_variables=['frute'], template='tell me the colore of {frute} in single word.')

In [8]:
llm = OpenAI(temperature=0)

In [9]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x0000018150A275F0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000018150DEA900>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [275]:
llm = OpenAI(temperature=0, model = 'davinci-002', verbose= True)
llm_chain = LLMChain(prompt=prompt, llm=llm)
#llm_chain.run(frute = "Apple")
llm_chain.invoke(input= "Apple")

{'frute': 'Apple',
 'text': ' I will tell you the name of the person who is your true love.\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true love?\n\nWhat is the name of the person who is your true lo

In [15]:
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['frute'], template='tell me the colore of {frute} in single word.'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x0000018150A275F0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000018150DEA900>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''))

In [ ]:
#llm_chain.run(frute = "Apple")
llm_chain.invoke(frute = "Apple")

In [19]:

template = """tell me the colore of {frute} in single word."""
prompt = PromptTemplate.from_template(template)
filled_prompt = prompt.format(frute ="orange" )
filled_prompt

'tell me the colore of orange in single word.'

In [20]:
llm.invoke(filled_prompt)

'\n\nOrange.'

# Tools how it works in langchain    how langchain used "Open-Meteo-API"

In [21]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools

In [ ]:
def _get_open_meteo_api(llm: BaseLanguageModel) -> BaseTool:
    chain = APIChain.from_llm_and_api_docs(
        llm,
        open_meteo_docs.OPEN_METEO_DOCS,
        limit_to_domains=["https://api.open-meteo.com/"],
    )
    return Tool(
        name="Open-Meteo-API",
        description="Useful for when you want to get weather information from the OpenMeteo API. The input should be a question in natural language that this API can answer.",
        func=chain.run,
    )

## API documentation

In [26]:
# flake8: noqa
#OPEN_METEO_DOCS = 
api_docs = """BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit	String	No	celsius	If fahrenheit is set, all temperature values are converted to Fahrenheit.
windspeed_unit	String	No	kmh	Other wind speed speed units: ms, mph and kn
precipitation_unit	String	No	mm	Other precipitation amount units: inch
timeformat	String	No	iso8601	If format unixtime is selected, all time values are returned in UNIX epoch time in seconds. Please note that all timestamp are in GMT+0! For daily values with unix timestamps, please apply utc_offset_seconds again to get the correct date.
timezone	String	No	GMT	If timezone is set, all timestamps are returned as local-time and data is returned starting at 00:00 local-time. Any time zone name from the time zone database is supported. If auto is set as a time zone, the coordinates will be automatically resolved to the local time zone.
past_days	Integer (0-2)	No	0	If past_days is set, yesterday or the day before yesterday data are also returned.
start_date
end_date	String (yyyy-mm-dd)	No		The time interval to get weather data. A day must be specified as an ISO8601 date (e.g. 2022-06-30).
models	String array	No	auto	Manually select one or more weather models. Per default, the best suitable weather models will be combined.

Hourly Parameter Definition
The parameter &hourly= accepts the following values. Most weather variables are given as an instantaneous value for the indicated hour. Some variables like precipitation are calculated from the preceding hour as an average or sum.

Variable	Valid time	Unit	Description
temperature_2m	Instant	°C (°F)	Air temperature at 2 meters above ground
snowfall	Preceding hour sum	cm (inch)	Snowfall amount of the preceding hour in centimeters. For the water equivalent in millimeter, divide by 7. E.g. 7 cm snow = 10 mm precipitation water equivalent
rain	Preceding hour sum	mm (inch)	Rain from large scale weather systems of the preceding hour in millimeter
showers	Preceding hour sum	mm (inch)	Showers from convective precipitation in millimeters from the preceding hour
weathercode	Instant	WMO code	Weather condition as a numeric code. Follow WMO weather interpretation codes. See table below for details.
snow_depth	Instant	meters	Snow depth on the ground
freezinglevel_height	Instant	meters	Altitude above sea level of the 0°C level
visibility	Instant	meters	Viewing distance in meters. Influenced by low clouds, humidity and aerosols. Maximum visibility is approximately 24 km."""

In [24]:
limit_to_domains=["https://api.open-meteo.com/"]

## prompt used for LLM calls

In [27]:
from langchain_core.prompts.prompt import PromptTemplate

API_URL_PROMPT_TEMPLATE = """You are given the below API Documentation:
{api_docs}
Using this documentation, generate the full API url to call for answering the user question.
You should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.

Question:{question}
API url:"""

API_URL_PROMPT = PromptTemplate(
    input_variables=[
        "api_docs",
        "question",
    ],
    template=API_URL_PROMPT_TEMPLATE,
)

API_RESPONSE_PROMPT_TEMPLATE = (
    API_URL_PROMPT_TEMPLATE
    + """ {api_url}

Here is the response from the API:

{api_response}

Summarize this response to answer the original question.

Summary:"""
)

API_RESPONSE_PROMPT = PromptTemplate(
    input_variables=["api_docs", "question", "api_url", "api_response"],
    template=API_RESPONSE_PROMPT_TEMPLATE,
)

## prompt1

In [30]:
print(API_URL_PROMPT.format(api_docs = api_docs, question = "what is the weather in Delhi today"))

You are given the below API Documentation:
BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit

## LLM response

In [39]:
api_response = "https://api.open-meteo.com/v1/forecast?latitude=28.7041&longitude=77.1025&hourly=temperature_2m,weathercode&current_weather=true"
api_response = "To get the weather in Delhi today, we need to include the latitude and longitude coordinates for Delhi. Since we're interested in the current weather conditions, we'll include the parameter `current_weather=true`. Here's the API URL:```https:/api.open-meteo.com/v1/forecast?latitude=28.7041&longitude=77.1025&current_weather=true``This URL will provide the current weather conditions for Delhi."


## prompt 2

In [31]:
API_RESPONSE_PROMPT_TEMPLATE

'You are given the below API Documentation:\n{api_docs}\nUsing this documentation, generate the full API url to call for answering the user question.\nYou should build the API url in order to get a response that is as short as possible, while still getting the necessary information to answer the question. Pay attention to deliberately exclude any unnecessary pieces of data in the API call.\n\nQuestion:{question}\nAPI url: {api_url}\n\nHere is the response from the API:\n\n{api_response}\n\nSummarize this response to answer the original question.\n\nSummary:'

In [40]:
print(API_RESPONSE_PROMPT.format(api_docs = api_docs,api_response = api_response, api_url = "https://api.open-meteo.com/",  question = "what is the weather in Delhi today" ))

You are given the below API Documentation:
BASE URL: https://api.open-meteo.com/

API Documentation
The API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:

Parameter	Format	Required	Default	Description
latitude, longitude	Floating point	Yes		Geographical WGS84 coordinate of the location
hourly	String array	No		A list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.
daily	String array	No		A list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.
current_weather	Bool	No	false	Include current weather conditions in the JSON output.
temperature_unit

## LLM response

In [ ]:
summary = "To retrieve the weather in Delhi today, we need to include the latitude and longitude coordinates for Delhi and specify that we want the current weather conditions. Here's the API URL: `https:/api.open-meteo.com/v1/forecast?latitude=28.7041&longitude=77.1025&current_weather=true`. This URL will provide the current weather conditions for Delhi."

## API fun call

In [41]:
import requests

# Define the API URL
api_url = "https://api.open-meteo.com/v1/forecast?latitude=28.7041&longitude=77.1025&current_weather=true"

# Make the GET request to the API
response = requests.get(api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Print the response content
    print(response.json())
else:
    # Print an error message if the request failed
    print("Error:", response.status_code)


{'latitude': 28.75, 'longitude': 77.125, 'generationtime_ms': 0.053048133850097656, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 222.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-04-11T06:00', 'interval': 900, 'temperature': 33.8, 'windspeed': 9.3, 'winddirection': 106, 'is_day': 1, 'weathercode': 2}}


## LLM call and final answer

sumarize the below weather details for delhi {'latitude': 28.75, 'longitude': 77.125, 'generationtime_ms': 0.053048133850097656, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 222.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-04-11T06:00', 'interval': 900, 'temperature': 33.8, 'windspeed': 9.3, 'winddirection': 106, 'is_day': 1, 'weathercode': 2}}. give your answer in one sentence

2 / 2

ChatGPT
The current weather in Delhi at 06:00 on April 11, 2024, includes a temperature of 33.8°C, windspeed of 9.3 km/h coming from 106°, with weather condition represented by WMO code 2.

# TOOLS custom calculator using LLMMathChain prompt not langchain tools

In [44]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools

## prompt1

In [45]:
# flake8: noqa
from langchain_core.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}
"""

PROMPT = PromptTemplate(
    input_variables=["question"],
    template=_PROMPT_TEMPLATE,
)

In [50]:
print(PROMPT.format(question ="What is (100*32)/300 + 20?" ))

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: What is (100*32)/300 + 20?



In [51]:
llm = OpenAI(temperature=0)
filled_prompt = PROMPT.format(question ="What is (100*32)/300 + 20?" )

llm_response = llm.invoke(filled_prompt)

In [91]:
llm_response = '```text\n(100*32)/300 + 20\n```\n...numexpr.evaluate("(100*32)/300 + 20")...\n```output\n52.666666666666664\n```\nAnswer: 52.666666666666664'
llm_response

'```text\n(100*32)/300 + 20\n```\n...numexpr.evaluate("(100*32)/300 + 20")...\n```output\n52.666666666666664\n```\nAnswer: 52.666666666666664'

## result explaination

In [66]:
print(llm_response)

```text
(100*32)/300 + 20
```
...numexpr.evaluate("(100*32)/300 + 20")...
```output
52.666666666666664
```
Answer: 52.666666666666664


In [70]:
llm_output = llm_response.strip()
import re
pattern = r"^```text\n(.*?)\n```"
text_match = re.search(pattern, llm_output, re.DOTALL)
text_match

<re.Match object; span=(0, 29), match='```text\n(100*32)/300 + 20\n```'>

In [71]:
if text_match:
    expression = text_match.group(1)
    print("Extracted expression:", expression)
else:
    print("No match found.")

Extracted expression: (100*32)/300 + 20


In [79]:
'          (100*32)/300 + 20      '.strip()

'(100*32)/300 + 20'

In [87]:
import numexpr
out = numexpr.evaluate(expression.strip())
re.sub(r"^\[|\]$", "", str(out)), out, str(out)

('30.666666666666664', array(30.66666667), '30.666666666666664')

In [144]:

import numexpr
out = numexpr.evaluate("13**(1.3432)".strip())
re.sub(r"^\[|\]$", "", str(out)), out, str(out)

('31.350806014579007', array(31.35080601), '31.350806014579007')

In [152]:
numexpr.evaluate("13**(1.3432)")

array(31.35080601)

## functions

In [94]:
def evaluate_expression( expression: str) -> str:
    import numexpr  # noqa: F401
    import math

    try:
        local_dict = {"pi": math.pi, "e": math.e}
        output = str(
            numexpr.evaluate(
                expression.strip(),
                global_dict={},  # restrict access to globals
                local_dict=local_dict,  # add common mathematical functions
            )
        )
    except Exception as e:
        raise ValueError(
            f'LLMMathChain._evaluate("{expression}") raised error: {e}.'
            " Please try again with a valid numerical expression"
        )

    # Remove any leading and trailing brackets from the output
    return re.sub(r"^\[|\]$", "", output)

In [95]:
def process_llm_result(
     llm_output: str, 
):
   
    llm_output = llm_output.strip()
    pattern = r"^```text\n(.*?)\n```"
    text_match = re.search(pattern, llm_output, re.DOTALL)
    if text_match:
        expression = text_match.group(1)
        output = evaluate_expression(expression)
        answer = "Answer: " + output
    elif llm_output.startswith("Answer:"):
        answer = llm_output
    elif "Answer:" in llm_output:
        answer = "Answer: " + llm_output.split("Answer:")[-1]
    else:
        raise ValueError(f"unknown format from LLM: {llm_output}")
    return answer

## final result

In [98]:
result = process_llm_result(llm_response)
result

'Answer: 30.666666666666664'

## prompt2 with davixchi model

In [158]:
# flake8: noqa
from langchain_core.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """Translate a math problem into a expression that can be executed using Python's numexpr library. 

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```

Question: 37593^(1/5)
```text
37593**(1/5)
```

Question: {question}
"""

PROMPT = PromptTemplate(
    input_variables=["question"],
    template=_PROMPT_TEMPLATE,
)


llm = OpenAI(temperature=0, model = 'davinci-002')
filled_prompt = PROMPT.format(question ="What is (100*pi)/e+ 20?" )  #(100*pi)/e+ 20  (100*32)/300 + 20 "What is 13 raised to the .3432 power?

llm_response = llm.invoke(filled_prompt,max_tokens=100)

In [115]:
print(llm_response)

```text
(100*32)/300 + 20
```

Question: What is 1000! / 1000!
```text
1000! / 1000!
```

Question: What is 1000! / 1000!
```text
1000! / 1000!
```

Question: What is 1000! / 1000!
```text
1000! / 1000!
```

Question: What is 1000


In [116]:
result = process_llm_result(llm_response)
result

'Answer: 30.666666666666664'

## response with devinchi without prompt

In [117]:
response = llm.invoke("What is (100*32)/300 + 20?",max_tokens=100)

' \n- How to deal with a nosy fellow graduate student? \n- How to objectively counter "You are the first one to complain about that."? \n- How to say something is easy? \n- Why would a recruiter want to put someone into a direct hire job at a lower salary than the stated salary range \n- How to design a GMPC that is not supposed to push the story ahead? \n- Why are films where the bad guy "wins" at the end so rare?'

In [122]:
print(response)

 
- How to deal with a nosy fellow graduate student? 
- How to objectively counter "You are the first one to complain about that."? 
- How to say something is easy? 
- Why would a recruiter want to put someone into a direct hire job at a lower salary than the stated salary range 
- How to design a GMPC that is not supposed to push the story ahead? 
- Why are films where the bad guy "wins" at the end so rare?


In [ ]:
llm = OpenAI(temperature=0, model = 'davinci-002')
filled_prompt = PROMPT.format(question ="What is (100*pi)/e+ 20?" )  #(100*pi)/e+ 20  (100*32)/300 + 20
llm_response = llm.invoke(filled_prompt,max_tokens=100)

In [119]:
print(llm_response)

```text
(100*pi)/e+ 20
```

Question: What is 2^3 * 3^2?
```text
2**3 * 3**2
```

Question: What is 2^3 * 3^2 + 2^3 * 3^2?
```text
2**3 * 3**2 + 2**3 * 3**2
```

Question: What is 2^3 * 


In [120]:
result = process_llm_result(llm_response)
result

'Answer: 135.57273497909216'

In [159]:

llm = OpenAI(temperature=0, model = 'davinci-002')
filled_prompt = PROMPT.format(question ="What is (100*pi)/e+ 20?" )  #(100*pi)/e+ 20  (100*32)/300 + 20
llm_response = llm.invoke(filled_prompt,stop=["```output"])
llm_response


'```text\n(100*pi)/e+ 20\n```\n\nQuestion: What is 2^3 * 3^2?\n```text\n2**3 * 3**2\n```\n\nQuestion: What is 2^3 * 3^2 + 2^3 * 3^2?\n```text\n2**3 * 3**2 + 2**3 * 3**2\n```\n\nQuestion: What is 2^3 * 3^2 + 2^3 * 3^2 - 2^3 * 3^2?\n```text\n2**3 * 3**2 + 2**3 * 3**2 - 2**3 * 3**2\n```\n\nQuestion: What is 2^3 * 3^2 + 2^3 * 3^2 - 2^3 * 3^2 * 2^3 * 3^2?\n```text\n2**3 * 3**2 + 2**3 * 3**2 - 2**3 * 3**2 * 2**3 * 3**2\n```\n\nQuestion: What is 2^3 * '

In [123]:
response = llm.invoke("What is (100*pi)/e+ 20?" ,max_tokens=100)

In [124]:
print(response)

 
- How to deal with a nosy fellow graduate student? 
- How to objectively counter "You are the first one to complain about that."? 
- How to say something is easy? 
- Why would a recruiter want to put someone into a direct hire job at a lower salary than the stated salary range 
- Why are films where the bad guy "wins" at the end so rare? 
- How to design a GMPC that is not supposed to push the story ahead?


## example

In [153]:
# flake8: noqa
from langchain_core.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """Translate a math problem into a expression that can be executed using Python's numexpr library. 

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```

Question: 37593^(1/5)
```text
37593**(1/5)
```

Question: {question}
"""

PROMPT = PromptTemplate(
    input_variables=["question"],
    template=_PROMPT_TEMPLATE,
)


llm = OpenAI(temperature=0, model = 'davinci-002')
filled_prompt = PROMPT.format(question ="What is 13 raised to the 0.3432 power?" )  #(100*pi)/e+ 20  (100*32)/300 + 20 "What is 13 raised to the .3432 power?

llm_response = llm.invoke(filled_prompt,max_tokens=100)

In [154]:
llm_response

'```text\n13**(0.3432)\n```\n\nQuestion: What is 13 raised to the 0.3432 power?\n```text\n13**(0.3432)\n```\n\nQuestion: What is 13 raised to the 0.3432 power?\n```text\n13**(0.3432)\n```\n\nQuestion: What is 13 raised to the 0.3432 power?\n```text\n13**(0.3432)\n```\n\nQuestion: What is'

In [155]:
result = process_llm_result(llm_response)
result

'Answer: 2.4116004626599237'

# LLMMathchain

In [131]:
from langchain.chains.llm_math.base import LLMMathChain

In [157]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x0000018152DC0680>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000018152E1ECF0>, model_name='davinci-002', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [156]:
llm_math = LLMMathChain.from_llm(llm, verbose=True)
llm_math.run("What is 13 raised to the .3432 power?")



> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13**(0.3432)
```
...numexpr.evaluate("13**(0.3432)")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

In [140]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



# Agents

In [172]:
from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import load_tools

prompt = hub.pull("hwchase17/react")


tools = load_tools([ "llm-math"], llm=llm)

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)
            
# agent_executor.invoke({"input": "hi"})

In [186]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

In [187]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [170]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [179]:
tools[0]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.resources.completions.Completions ob

In [180]:
print('Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [183]:
print(prompt.format(tools ="Calculator: Useful for when you need to answer questions about math.", tool_names = 'Calculator', input = "what is two plus two" , agent_scratchpad ="" ))

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: what is two plus two
Thought:


In [173]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Calculator: Useful for when you need to answer questions about math.', 'tool_names': 'Calculator'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final an

<bound method Runnable.get_prompts of RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'Calculator: Useful for when you need to answer questions about math.', 'tool_names': 'Calculator'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer:

## custom agent

In [204]:
from langchain.tools import Tool
# from langchain.agents import tool

#@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return str(len(word))

In [223]:
Tool.from_function(
    func=get_word_length,
    name ="word lenght calculator",
    description="calculate the length of word"
)

Tool(name='word lenght calculator', description='calculate the length of word', func=<function get_word_length at 0x000001815383C400>)

In [225]:
from langchain.tools import BaseTool
from math import pi
from typing import Union

class Get_word_length(BaseTool):
    name = "word lenght calculator"
    description = "calculate the length of word"

    def _run(self, word: str) -> str:
        return str(len(word))

    async def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
    

# class CircumferenceTool(BaseTool):
#     name = "Circumference calculator"
#     description = "Use this tool when you need to calculate a circumference using the radius of a circle"

#     def _run(self, radius: Union[int, float]) -> float:
#         return float(radius) * 2.0 * pi

#     async def _arun(self, radius: int):
#         raise NotImplementedError("This tool does not support async")


In [226]:
from langchain_openai import OpenAI
llm = OpenAI()
template= "how many letters ara in {content}"
prompt = PromptTemplate.from_template(template)
prompt.format(content= "APPLE")

'how many letters ara in APPLE'

In [232]:
from langchain.agents import initialize_agent, AgentType

tools = [Get_word_length()]


agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)

In [233]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

word lenght calculator: calculate the length of word

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [word lenght calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [234]:
agent.run('how many letters ara in APPLE')



> Entering new AgentExecutor chain...
 Count the number of letters
Action: word length calculator
Action Input: APPLE
Observation: word length calculator is not a valid tool, try one of [word lenght calculator].
Thought: I need to double check the spelling
Action: check the spelling
Action Input: APPLE
Observation: check the spelling is not a valid tool, try one of [word lenght calculator].
Thought: I need to use a different tool
Action: word length calculator
Action Input: APPLE
Observation: word length calculator is not a valid tool, try one of [word lenght calculator].
Thought:

KeyboardInterrupt: 

## example

In [247]:
from langchain.tools import BaseTool
from math import pi
from typing import Union
  

class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "Use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]) -> float:
        return float(radius) * 2.0 * pi

  


from langchain.agents import initialize_agent

tools = [CircumferenceTool()]

# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,)


In [248]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Circumference calculator: Use this tool when you need to calculate a circumference using the radius of a circle

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Circumference calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [249]:
agent("can you calculate the circumference of a circle that has a radius of 7.81")



> Entering new AgentExecutor chain...
 I need to use the circumference calculator to find the circumference
Action: Circumference calculator
Action Input: 7.81
Observation: 49.071677249072565
Thought: This is the circumference of the circle with a radius of 7.81
Final Answer: The circumference of the circle with a radius of 7.81 is 49.071677249072565.

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81',
 'output': 'The circumference of the circle with a radius of 7.81 is 49.071677249072565.'}

In [246]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Circumference calculator: Use this tool when you need to calculate a circumference using the radius of a circle

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Circumference calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [271]:
llm = OpenAI(verbose=True)
llm.invoke("what is temperature today in Delhi?", stop=["."])

'\n\nI am an AI and do not have access to real-time data'

In [252]:
import requests
import json
from datetime import date, datetime, timedelta
import os

from typing import Optional, Dict, Union, List

class WeatherTeller(BaseTool):
    name = "Weather forcaster"
    description = "Use this tool when you need weather information of any city"

    def _run(self, city: str):
        url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

        querystring = {}
        
        if city:
            querystring['city'] = city

        headers = {
            "X-RapidAPI-Key": "f5ce08b98fmsh493ce5f935f768fp113ce0jsn156ad2cdf9db",
            "X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        try:
            observation = response.json()
        except:
            observation = response.text
        return observation


from langchain.agents import initialize_agent

tools = [WeatherTeller()]

# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,)

In [253]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Weather forcaster: Use this tool when you need weather information of any city

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Weather forcaster]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [254]:
agent("what is weather today in Delhi?")



> Entering new AgentExecutor chain...
 I should check the weather forecast for Delhi
Action: Weather forcaster
Action Input: Delhi
Observation: {'cloud_pct': 38, 'temp': 37, 'feels_like': 34, 'humidity': 13, 'min_temp': 36, 'max_temp': 37, 'wind_speed': 5.14, 'wind_degrees': 50, 'sunrise': 1712795374, 'sunset': 1712841257}
Thought: I should check the current temperature and humidity
Action: Weather forcaster
Action Input: Delhi
Observation: {'cloud_pct': 38, 'temp': 37, 'feels_like': 34, 'humidity': 13, 'min_temp': 36, 'max_temp': 37, 'wind_speed': 5.14, 'wind_degrees': 50, 'sunrise': 1712795374, 'sunset': 1712841257}
Thought: The current temperature in Delhi is 37 degrees with a humidity of 13%
Final Answer: The weather today in Delhi is partly cloudy with a high of 37 degrees and a low of 36 degrees. The humidity is at 13%.

> Finished chain.


{'input': 'what is weather today in Delhi?',
 'output': 'The weather today in Delhi is partly cloudy with a high of 37 degrees and a low of 36 degrees. The humidity is at 13%.'}

In [257]:
import requests
import json
from datetime import date, datetime, timedelta
import os

from typing import Optional, Dict, Union, List

class WeatherTeller(BaseTool):
    name = "Weather forcaster"
    description = "Use this tool when you need weather information of any city"

    def _run(self, city: str):
        url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

        querystring = {}
        
        if city:
            querystring['city'] = city

        headers = {
            "X-RapidAPI-Key": "f5ce08b98fmsh493ce5f935f768fp113ce0jsn156ad2cdf9db",
            "X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        try:
            observation = response.json()
        except:
            observation = response.text
        return observation


from langchain.agents import initialize_agent

tools = [WeatherTeller()]

# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,)
llm = OpenAI(temperature=0, model = 'davinci-002')

agent("what is temperature today in Delhi?")



> Entering new AgentExecutor chain...
 I need to know the temperature in Delhi
Action: Weather forcaster
Action Input: Delhi
Observation: {'cloud_pct': 38, 'temp': 36, 'feels_like': 33, 'humidity': 14, 'min_temp': 36, 'max_temp': 36, 'wind_speed': 5.14, 'wind_degrees': 50, 'sunrise': 1712795374, 'sunset': 1712841257}
Thought: I now know the temperature in Delhi
Final Answer: 36

> Finished chain.


{'input': 'what is temperature today in Delhi?', 'output': '36'}

In [ ]:
agent.run("what is temperature today in Delhi?")

# debugging the AGENT TOOL interaction

In [269]:
from langchain.callbacks import StdOutCallbackHandler
tools = [WeatherTeller()]
llm = OpenAI(temperature=0, model = 'davinci-002',verbose=True)
# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,)



agent.agent.llm_chain.verbose=True
callbacks = [StdOutCallbackHandler()]
agent.run("what is temperature today in Delhi?", callbacks=callbacks)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Weather forcaster: Use this tool when you need weather information of any city

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Weather forcaster]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: what is temperature today in Delhi?
Thought:

> Finished chain.
 I need to know the temperature in Delhi
Action: Weather forcaster
Action Input: Delhi
Observation: {'cloud_pct': 25, 'temp': 36, 'feels_like': 33, 'humidity': 14, 'min_temp': 36, 'max_temp': 36, 'wind_speed': 4.63, 

'36'

In [ ]:
from langchain.callbacks import StdOutCallbackHandler
tools = [WeatherTeller()]
llm = OpenAI(temperature=0, model = 'davinci-002',verbose=True)
# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,)



agent.agent.llm_chain.verbose=True
callbacks = [StdOutCallbackHandler()]
agent.run("what is temperature today in Delhi?", callbacks=callbacks)

In [ ]:

from langchain.callbacks import StdOutCallbackHandler, WandbCallbackHandler
tools = [WeatherTeller()]
llm = OpenAI(temperature=0, model = 'davinci-002',verbose=True)


# Weights & Biases
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
wandb_callback = WandbCallbackHandler(
    job_type="inference",
    project="LangChain Demo",
    group=f"minimal_{session_group}",
    name="llm",
    tags=["demo"],
)

callbacks = [StdOutCallbackHandler(), wandb_callback]



# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,)



agent.agent.llm_chain.verbose=True

agent.run("what is temperature today in Delhi?", callbacks=callbacks)

wandb_callback.flush_tracker(agent, reset=False, finish=True)

In [ ]:
!pip install pip install wandb